<a href="https://colab.research.google.com/github/Smpests/KeepLearning/blob/master/jupyter-notebook/sqlalchemy_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sqlalchemy import create_engine, Table, Column, Integer, String, DATETIME, Float, VARCHAR, TEXT, Index
from sqlalchemy.orm import declarative_base, Session, relationship

# echo参数会打印回显到标准输出
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
Base = declarative_base()

class User(Base):
  __tablename__ = "user"

  id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String(30), index=True, nullable=False, doc={"zh": "名字"})
  create_time = Column(DATETIME, doc={'zh': '创建日期'})
  fullname = Column(String)
  # relationship需要使用外键，不推荐
  # addresses = relationship("Address", back_populates="user")
  # Index("索引名称", ...列名)
  __table_args__ = (Index("NAME_FULLNAME_INDEX", "name", "fullname"), )

  def __repr__(self):
    # !r表示该表量调用其__repr__方法
    return f"User(id={self.id!r}, name={self.name!r}, fullname={self.fullname!r})"

class Address(Base):
  __tablename__ = "address"

  id = Column(Integer, primary_key=True, autoincrement=True)
  name = Column(String)
  # user = relationship("User", back_populates="addresses")


sandy = User(name="sandy", fullname="Sandu Cheeks")
print(sandy)
# 创建所有Base子类的表
Base.metadata.create_all(engine)

In [33]:
# insert demo
from sqlalchemy import insert

stmt = insert(User).values(name="spongebob", fullname="Spongebob Squarepants")
with Session(engine) as session:
  result1 = session.execute(stmt)
  session.commit()

print(result1.inserted_primary_key)



2023-04-04 11:22:45,998 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-04 11:22:46,003 INFO sqlalchemy.engine.Engine INSERT INTO user (name, fullname) VALUES (?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO user (name, fullname) VALUES (?, ?)


2023-04-04 11:22:46,005 INFO sqlalchemy.engine.Engine [generated in 0.00244s] ('spongebob', 'Spongebob Squarepants')


INFO:sqlalchemy.engine.Engine:[generated in 0.00244s] ('spongebob', 'Spongebob Squarepants')


2023-04-04 11:22:46,008 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


(1,)


In [34]:
# 插入多条数据
with Session(engine) as session:
  result2 = session.execute(
      insert(User),
      [
          {"name": "sandy", "fullname": "Sandy Cheek"},
          {"name": "patrick", "fullname": "Parick Star"},
      ]
  )
  session.commit()

2023-04-04 11:22:52,761 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-04 11:22:52,765 INFO sqlalchemy.engine.Engine INSERT INTO user (name, fullname) VALUES (?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO user (name, fullname) VALUES (?, ?)


2023-04-04 11:22:52,766 INFO sqlalchemy.engine.Engine [generated in 0.00195s] (('sandy', 'Sandy Cheek'), ('patrick', 'Parick Star'))


INFO:sqlalchemy.engine.Engine:[generated in 0.00195s] (('sandy', 'Sandy Cheek'), ('patrick', 'Parick Star'))


2023-04-04 11:22:52,768 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


In [45]:
from sqlalchemy import select
# 查询
with Session(engine) as session:
  # 不指定table.column则输出全部
  rows = session.execute(select(User.name, User.fullname)).first()
  rows2 = session.query(User).where(User.id > 1).first()
print(rows)
print(rows2)

2023-04-04 11:54:42,910 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-04 11:54:42,914 INFO sqlalchemy.engine.Engine SELECT user.name, user.fullname 
FROM user


INFO:sqlalchemy.engine.Engine:SELECT user.name, user.fullname 
FROM user


2023-04-04 11:54:42,920 INFO sqlalchemy.engine.Engine [cached since 1563s ago] ()


...

2023-04-04 11:54:42,933 INFO sqlalchemy.engine.Engine [generated in 0.00246s] (1, 1, 0)


INFO:sqlalchemy.engine.Engine:[generated in 0.00246s] (1, 1, 0)


2023-04-04 11:54:42,936 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


('patrick', 'Parick Star')
User(id=2, name='sandy', fullname='Sandy Cheek')


In [46]:
squidward = User(name="squidward", fullname="Squidward Tentacles")
_session = Session(engine)
_session.add(squidward)
print(squidward.id)
# flush之后就有id了
_session.flush()
print(squidward.id)
# 不commit或者rollback，事务一直开启
# session.commit()

None
2023-04-04 12:03:56,859 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-04 12:03:56,864 INFO sqlalchemy.engine.Engine INSERT INTO user (name, create_time, fullname) VALUES (?, ?, ?)


INFO:sqlalchemy.engine.Engine:INSERT INTO user (name, create_time, fullname) VALUES (?, ?, ?)


2023-04-04 12:03:56,868 INFO sqlalchemy.engine.Engine [generated in 0.00396s] ('squidward', None, 'Squidward Tentacles')


INFO:sqlalchemy.engine.Engine:[generated in 0.00396s] ('squidward', None, 'Squidward Tentacles')


4


In [52]:
# 更新数据
sandy = session.execute(select(User).filter_by(name="sandy")).scalar_one()
print(sandy)

sandy.fullname = "Sandy Update"
# sandy更改过之后会被标记为dirty
print(sandy in session.dirty)
# 执行flush()或者任意查询都会自动flush一次
# session.flush()
# 返回的将是修改后的值
new_sandy = session.query(User).where(User.id == 2).scalar()
print(new_sandy)
print(sandy in session.dirty)


2023-04-04 13:00:56,957 INFO sqlalchemy.engine.Engine SELECT user.id, user.name, user.create_time, user.fullname 
FROM user 
WHERE user.name = ?


INFO:sqlalchemy.engine.Engine:SELECT user.id, user.name, user.create_time, user.fullname 
FROM user 
WHERE user.name = ?


2023-04-04 13:00:56,963 INFO sqlalchemy.engine.Engine [cached since 2938s ago] ('sandy',)


INFO:sqlalchemy.engine.Engine:[cached since 2938s ago] ('sandy',)


User(id=2, name='sandy', fullname='Sandy Sample')
True
2023-04-04 13:00:56,972 INFO sqlalchemy.engine.Engine UPDATE user SET fullname=? WHERE user.id = ?


...

2023-04-04 13:00:56,982 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.name AS user_name, user.create_time AS user_create_time, user.fullname AS user_fullname 
FROM user 
WHERE user.id = ?


INFO:sqlalchemy.engine.Engine:SELECT user.id AS user_id, user.name AS user_name, user.create_time AS user_create_time, user.fullname AS user_fullname 
FROM user 
WHERE user.id = ?


2023-04-04 13:00:56,984 INFO sqlalchemy.engine.Engine [cached since 33.6s ago] (2,)


INFO:sqlalchemy.engine.Engine:[cached since 33.6s ago] (2,)


User(id=2, name='sandy', fullname='Sandy Update')
False


In [54]:
from sqlalchemy import update, delete
session.execute(update(User).where(User.id == 2).values(fullname="Sandy Azure"))
# 更新之后之前的绑定对象也会改变
print(sandy)

2023-04-04 13:06:36,167 INFO sqlalchemy.engine.Engine UPDATE user SET fullname=? WHERE user.id = ?


INFO:sqlalchemy.engine.Engine:UPDATE user SET fullname=? WHERE user.id = ?


2023-04-04 13:06:36,174 INFO sqlalchemy.engine.Engine [generated in 0.00634s] ('Sandy Azure', 2)


INFO:sqlalchemy.engine.Engine:[generated in 0.00634s] ('Sandy Azure', 2)


User(id=2, name='sandy', fullname='Sandy Azure')


In [62]:
# get()函数通过主键获取数据
patrick = session.get(User, 3)
print(patrick)
print(patrick in session)
# 删除数据
session.delete(patrick)
# session.execute(delete(User).where(User.id == 3))
session.query(User).where(User.name == "patrick").first()
print(patrick)
print(patrick in session)


User(id=3, name='patrick', fullname='Parick Star')
True
2023-04-04 13:13:02,171 INFO sqlalchemy.engine.Engine DELETE FROM user WHERE user.id = ?


INFO:sqlalchemy.engine.Engine:DELETE FROM user WHERE user.id = ?


2023-04-04 13:13:02,178 INFO sqlalchemy.engine.Engine [cached since 127.7s ago] (3,)


INFO:sqlalchemy.engine.Engine:[cached since 127.7s ago] (3,)


2023-04-04 13:13:02,181 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.name AS user_name, user.create_time AS user_create_time, user.fullname AS user_fullname 
FROM user 
WHERE user.name = ?
 LIMIT ? OFFSET ?


<ipython-input-62-880df1e16a3f>:7: SAWarning: DELETE statement on table 'user' expected to delete 1 row(s); 0 were matched.  Please set confirm_deleted_rows=False within the mapper configuration to prevent this warning.
  session.query(User).where(User.name == "patrick").first()
INFO:sqlalchemy.engine.Engine:SELECT user.id AS user_id, user.name AS user_name, user.create_time AS user_create_time, user.fullname AS user_fullname 
FROM user 
WHERE user.name = ?
 LIMIT ? OFFSET ?


2023-04-04 13:13:02,185 INFO sqlalchemy.engine.Engine [cached since 44.81s ago] ('patrick', 1, 0)


INFO:sqlalchemy.engine.Engine:[cached since 44.81s ago] ('patrick', 1, 0)


User(id=3, name='patrick', fullname='Parick Star')
False


In [63]:
# 执行rollback上面进行的update和delete操作都会回退
session.rollback()
print(sandy)
print(patrick in session)

2023-04-04 13:15:20,867 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


2023-04-04 13:15:20,875 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2023-04-04 13:15:20,881 INFO sqlalchemy.engine.Engine SELECT user.id AS user_id, user.name AS user_name, user.create_time AS user_create_time, user.fullname AS user_fullname 
FROM user 
WHERE user.id = ?


INFO:sqlalchemy.engine.Engine:SELECT user.id AS user_id, user.name AS user_name, user.create_time AS user_create_time, user.fullname AS user_fullname 
FROM user 
WHERE user.id = ?


2023-04-04 13:15:20,885 INFO sqlalchemy.engine.Engine [generated in 0.00313s] (2,)


INFO:sqlalchemy.engine.Engine:[generated in 0.00313s] (2,)


User(id=2, name='sandy', fullname='Sandy Cheek')
True


In [64]:
# 操作完毕后关闭session，可以用with关键字自动关闭
print(sandy in session)
# close时会执行rollback
session.close()
print(sandy in session)


True
2023-04-04 13:28:55,776 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK


False


In [ ]:
# 重新绑定
session.add(sandy)
sandy.name
# session关闭后未绑定的ORM实例将无法获取值
patrick.name